In [1]:
import pandas as pd
import glob
from matplotlib import pyplot
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import warnings

from collections import Counter
from tqdm import tqdm
import os
import math

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# function that finds the indexes of non-anomalies for interpolation 
def interpolation_indexes(mylist, mynumber):
    
    left_neighbour = 0
    right_neighbour = 0
    
    # check left neighbour
    if((mynumber - 1) not in mylist):
        left_neighbour = mynumber - 1
    else:
        min_number = mynumber
        while min_number in mylist:
            min_number = min_number - 1
        left_neighbour = min_number
    
    # check right neighbour
    if((mynumber + 1) not in mylist):
        right_neighbour = mynumber + 1
    else:
        max_number = mynumber
        while max_number in mylist:
            max_number = max_number + 1
        right_neighbour = max_number
    
    return left_neighbour, right_neighbour

In [4]:
def lstm_model():
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
            kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(8, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X_train.shape[1])(L2)
    L4 = LSTM(8, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X_train.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='mae')
    return model

In [5]:
def train_anomaly_removal(df_train):
    
    # extract indexes for anomalies
    indexes = list(df_train[df_train.is_anomaly == 1].index)

    # creating a new df that replaces the anomalous samples with interpolation value
    df = pd.DataFrame(columns = df_train.columns)
    for i in range(0, len(df_train)):

        #print(i)

        # add all non-anomalies
        if(df_train.is_anomaly[i] == 0):
            df = df.append({'timestamp' : df_train.timestamp[i], 'value' : df_train.value[i], 'is_anomaly' : df_train.is_anomaly[i]},
            ignore_index = True)

        if((df_train.is_anomaly[i]==1) & (i != (len(df_train)-1)) & (i != 0)):
            if(df_train.is_anomaly[i+1]!=1):
                
                if((interpolation_indexes(indexes, i)[0] != -1) & (interpolation_indexes(indexes, i)[1] != -1)):
                    value_interpolation = (df_train.value[interpolation_indexes(indexes, i)[0]]
                                           +df_train.value[interpolation_indexes(indexes, i)[1]])/2

                    df = df.append({'timestamp' : df_train.timestamp[i], 'value': value_interpolation, 'is_anomaly' : 0.0}, ignore_index = True)
    return df

In [6]:
def threshold_computing_max(X_train):
    X_train_pred = model.predict(X_train, verbose=0)
    train_mae_loss_avg = np.mean(np.abs(X_train_pred - X_train), axis=1)
    max_threshold = np.max(train_mae_loss_avg)
    return max_threshold

In [7]:
def reconstruction_loss_predictions(X_test):
    X_test_pred = model.predict(X_test, verbose=0)
    mae_loss = np.mean(np.abs(X_test_pred-X_test), axis=1)
    return mae_loss, X_test_pred

In [8]:
def predicted_labels(mae_loss, threshold):
    predicted_test_label = []
    for i in range(0, len(test_mae_loss)):
        if(test_mae_loss[i][0]>(threshold)):
            predicted_test_label.append(1)
        else:
            predicted_test_label.append(0)
    return predicted_test_label

In [9]:
# Yahoo
#path_files = '../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/Yahoo_A1Benchmark/'
# NAB
path_files = '../../../Documents/phd_related/data_sets_concept_drift/anomaly_detection/NAB/'

## Extract all file names corresponding to time series

In [10]:
ts_names = []
for i in os.listdir(path_files):
    ts_names.append(str(i.split('.csv')[0]))

In [15]:
df_final_results_details = pd.DataFrame(columns = ['TS_name', 'lstmae_reconstruction_loss'])
df_final_results = pd.DataFrame(columns = ['TS_name', 'Labels_True', 'Labels_Pred', 'Test_Size', 'Model'])

scaler = MinMaxScaler()

# for daily retraining NAB
#window_retraining = 288

# for weekly retraining NAB
window_retraining = 2016

# for weekly retraining Yahoo
#window_retraining = 168

for ts_name in tqdm(ts_names):
    for n in range(0, 5):

        label_pred_complete = []
        losses_complete = []

        print(ts_name)
        # path to train/test
        filename = path_files+ts_name+".csv"

        # read ts
        df = pd.read_csv(filename)


        # split into train and test
        init_train = df[0:math.floor(len(df)/2)]
        # print(len(init_train))
        init_test = df[math.floor(len(df)/2):]
        # print(len(init_test))


        for i in tqdm(range(0, (math.floor(len(init_test)/window_retraining)+1))):

            # adjust training over time
            df_train = pd.concat([init_train, init_test[0:window_retraining*i]], ignore_index=True)


            # adjust testing over time

            if(i == (round(len(init_test)/window_retraining))):
                df_test = init_test[(i)*window_retraining:]
            else:
                df_test = init_test[(i*window_retraining):((i+1)*window_retraining)]


            # remove anomalies from train to prepare LSTM
            # all anomalies are replaced by the interpolation of their closest non-anomalous neighbours
            df_train = train_anomaly_removal(df_train)


            # final training dataset + labels
            label_train = df_train.is_anomaly
            train = df_train.value



            # final testing dataset + labels
            label_test = df_test.is_anomaly
            test = df_test.value
            
            # in case the training contains all data and there is no more data left for testing
            if(df_test.empty):
                break


            # Data preprocessing - Scaling
            # the scaler is fit on the training data and applied on the testing data
            train_scale = scaler.fit_transform(np.array(train).reshape(-1, 1))
            test_scale = scaler.transform(np.array(test).reshape(-1,1))

            # Shape Train Data for LSTM
            X_train = train_scale.reshape(train_scale.shape[0], 1, 1)

            # Train LSTM
            no_epochs = 50
            batch_size = 128
            model = lstm_model()
            encdec = model.fit(X_train, X_train, epochs=no_epochs, batch_size=batch_size,
                                validation_split=0.25).history

            # Threshold computing
            threshold = threshold_computing_max(X_train)

            # Shape Test Data for LSTM
            X_test = test_scale.reshape(test_scale.shape[0], 1, 1)

            test_mae_loss, X_test_pred = reconstruction_loss_predictions(X_test)

            # Extracting Predicted Labels
            y_label_pred = predicted_labels(test_mae_loss, threshold)

            label_pred_complete.append(y_label_pred)
            losses_complete.append(threshold)

        all_predicted_labels = []
        for i in range(0, len(label_pred_complete)):
            for j in range(0, len(label_pred_complete[i])):
                all_predicted_labels.append(label_pred_complete[i][j])




        # Save Results
        # Save reconstruction Error for each Dataset
        df_results_details = pd.DataFrame()
        df_results_details['TS_name'] = [ts_name]
        df_results_details['lstmae_reconstruction_loss'] = [losses_complete]
        df_results_details['retraining_technique'] = 'full_history'
        df_results_details['retraining_window'] = window_retraining
        df_final_results_details = df_final_results_details.append(df_results_details)


        # Save Predicted Labels
        df_results = pd.DataFrame()


        df_results['TS_name'] = [ts_name]
        df_results['retraining_technique'] = 'full_history'
        df_results['retraining_window'] = window_retraining
        df_results['Labels_True'] = [list(init_test.is_anomaly)]
        df_results['Labels_Pred'] = [all_predicted_labels]
        df_results['Test_Size'] = len(list(init_test.is_anomaly))
        df_results['Model'] = 'LSTM_AE'
        df_final_results = df_final_results.append(df_results)


  0%|          | 0/2 [00:00<?, ?it/s]

ec2_network_in_257a54
Epoch 1/50
12/12 [==============================] - 3s 33ms/step - loss: 0.0023 - val_loss: 0.0045
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 3/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 4/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 5/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 6/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 8/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 9/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 10/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0


 50%|█████     | 1/2 [00:22<00:22, 22.15s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

ec2_network_in_257a54
Epoch 1/50
12/12 [==============================] - 3s 34ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 3/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 4/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 5/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 6/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 8/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 9/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0045
Epoch 10/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0


  0%|          | 0/1 [00:34<?, ?it/s]


KeyboardInterrupt: 

In [16]:
df_final_results = df_final_results.reset_index(drop=True)
df_final_results_details = df_final_results_details.reset_index(drop=True)

In [ ]:
#df_final_results.to_csv('./results/df_results_lstmae_periodic_fh.csv')